## cross dataset analysis with covid19 datasets

#### analyze covid19 cases worldwide

In [1]:
%%bigquery
select date, country_region, sum(confirmed) as confirmed, sum(deaths) as deaths, sum(recovered) as recovered, sum(active) as active
from covid19_jhu_csse_modeled.reports r join covid19_jhu_csse_modeled.location_id_Beam_DF l
on r.location_id = l.id
group by date, country_region
order by date desc
limit 12

,date,country_region,confirmed,deaths,recovered,active
0,2020-05-02,Bahamas,83,11,24,48
1,2020-05-02,Libya,63,3,22,38
2,2020-05-02,Australia,6799,94,5814,891
3,2020-05-02,Sudan,592,41,52,499
4,2020-05-02,Tajikistan,76,2,0,74
5,2020-05-02,Indonesia,10843,831,1665,8347
6,2020-05-02,France,168518,24763,50663,93092
7,2020-05-02,Guyana,82,9,22,51
8,2020-05-02,United States,1132539,66369,175382,917894
9,2020-05-02,Japan,14571,474,3205,10892


analyze mobility changes worldwide

In [2]:
%%bigquery
select date, country_region, avg(retail_and_rec) as retail_and_rec, avg(grocery_and_pharm) as grocery_and_pharm, avg(parks) as parks, avg(transit_stations) as transit_stations, avg(workplaces) as workplaces, avg(residential) as residential
from google_modeled.mobility_event g join google_modeled.location l
on g.location_id = l.id
group by date, country_region
order by date desc
limit 12

,date,country_region,retail_and_rec,grocery_and_pharm,parks,transit_stations,workplaces,residential
0,2020-04-26,United Arab Emirates,-49.375000,-24.750000,-66.000000,-59.428571,-41.875000,26.857143
1,2020-04-26,New Zealand,-90.400000,-38.466667,-72.647059,-81.083333,-43.941176,20.000000
2,2020-04-26,Uruguay,-65.384615,-39.800000,-73.850000,-66.882353,-22.684211,19.000000
3,2020-04-26,United States,-31.938215,-10.985950,4.608637,-31.217259,-30.369245,9.894891
4,2020-04-26,Kenya,-47.727273,-39.923077,-28.250000,-42.320000,-16.678571,21.000000
5,2020-04-26,Hungary,-43.150000,-24.952381,22.800000,-32.952381,-19.476190,5.562500
6,2020-04-26,Slovenia,-80.100000,-89.500000,-30.666667,-52.400000,-33.857143,9.500000
7,2020-04-26,Portugal,-76.952381,-50.285714,-65.476190,-74.375000,-42.190476,19.363636
8,2020-04-26,Bulgaria,-52.894737,-24.227273,-13.650000,-45.181818,-17.172414,6.875000
9,2020-04-26,United Kingdom,-77.368794,-35.621622,-17.530769,-55.687943,-45.979866,13.460317


cross dataset queries

In [3]:
%%bigquery
select j1.date, j1.country_region, confirmed, workplaces, residential from 
(select r.date, l.country_region, sum(confirmed) as confirmed
from covid19_jhu_csse_modeled.reports r 
join covid19_jhu_csse_modeled.location_id_Beam_DF l on r.location_id = l.id
group by r.date, country_region) j1
join (select g.date, gl.country_region_code, gl.country_region, avg(workplaces) as workplaces, avg(residential) as residential from google_modeled.mobility_event g
join google_modeled.location gl on gl.id = g.location_id
group by g.date, country_region_code, country_region) j2
on j1.date = j2.date and j1.country_region = j2.country_region
order by date desc, country_region
limit 12

,date,country_region,confirmed,workplaces,residential
0,2020-04-26,Afghanistan,1531,-37.000000,17.000000
1,2020-04-26,Angola,26,-9.000000,19.000000
2,2020-04-26,Antigua and Barbuda,24,-53.000000,NaN
3,2020-04-26,Argentina,3892,-42.080000,24.240000
4,2020-04-26,Australia,6714,-30.111111,11.777778
5,2020-04-26,Austria,15225,-34.600000,8.111111
6,2020-04-26,Bahrain,2647,-24.000000,19.000000
7,2020-04-26,Bangladesh,5416,-56.000000,25.000000
8,2020-04-26,Barbados,79,-58.800000,17.000000
9,2020-04-26,Belarus,10463,-9.000000,0.000000


#### analyze US stats

In [4]:
%%bigquery
select date, sum(confirmed) as confirmed, sum(deaths) as deaths, sum(recovered) as recovered, sum(active) as active
from covid19_jhu_csse_modeled.reports r join covid19_jhu_csse_modeled.location_id_Beam_DF l
on r.location_id = l.id
where country_region = 'United States' 
group by date
order by date desc
limit 5

,date,confirmed,deaths,recovered,active
0,2020-05-02,1132539,66369,175382,917894
1,2020-05-01,1103461,64943,164015,903178
2,2020-04-30,1069424,62996,153947,878169
3,2020-04-29,1039909,60967,120720,858222
4,2020-04-28,1012582,58355,115936,838291


In [5]:
%%bigquery
select date, avg(retail_and_rec) as retail_and_rec, avg(grocery_and_pharm) as grocery_and_pharm, avg(parks) as parks, avg(transit_stations) as transit_stations, avg(workplaces) as workplaces, avg(residential) as residential
from google_modeled.mobility_event g join google_modeled.location l
on g.location_id = l.id
where country_region_code = 'US' 
group by date
order by date desc
limit 5

,date,retail_and_rec,grocery_and_pharm,parks,transit_stations,workplaces,residential
0,2020-04-26,-31.938215,-10.985950,4.608637,-31.217259,-30.369245,9.894891
1,2020-04-25,-32.901251,-6.931497,18.922652,-27.582016,-25.842702,11.154839
2,2020-04-24,-32.301352,-10.066854,1.854951,-30.052910,-37.474558,18.543513
3,2020-04-23,-32.612216,-9.902439,-4.493741,-29.091985,-38.722934,18.236722
4,2020-04-22,-29.921324,-8.738187,3.526395,-26.725114,-38.433808,17.022523


cross dataset queries

In [6]:
%%bigquery
select j1.date, j1.state, j1.confirmed, j1.deaths from
(select date, province_state as state, sum(j.confirmed) as confirmed, sum(j.deaths) as deaths
from covid19_jhu_csse_modeled.reports j 
join covid19_jhu_csse_modeled.us_location_id_Beam_DF l on j.location_id = l.id
group by date, province_state) j1
join
(select date, state, sum(confirmed_cases) as confirmed, sum(deaths) as deaths
from covid19_usafacts_modeled.reports r 
join covid19_usafacts_modeled.loc_state_Beam_DF s on s.state_fips_code = r.state_fips_code
group by date, state) j2 on j1.date=j2.date and j1.state=j2.state
order by date desc, state
limit 12

,date,state,confirmed,deaths
0,2020-05-02,Alabama,7611,288
1,2020-05-02,Alaska,365,9
2,2020-05-02,Arizona,8364,330
3,2020-05-02,Arkansas,3372,72
4,2020-05-02,California,53347,2180
5,2020-05-02,Colorado,16225,832
6,2020-05-02,Connecticut,29287,2436
7,2020-05-02,Delaware,5038,168
8,2020-05-02,District of Columbia,4797,240
9,2020-05-02,Florida,35463,1364


#### look at TX stats

In [7]:
%%bigquery
select date, sum(confirmed) as confirmed, sum(deaths) as deaths, sum(recovered) as recovered, sum(active) as active
from covid19_jhu_csse_modeled.reports r join covid19_jhu_csse_modeled.location_id_Beam_DF l
on r.location_id = l.id
where province_state  = 'Texas' or province_state like '%TX%'
group by date
order by date desc
limit 5

,date,confirmed,deaths,recovered,active
0,2020-05-02,30917,863,0,30054
1,2020-05-01,29692,840,0,28852
2,2020-04-30,28727,812,0,27915
3,2020-04-29,27257,754,0,26503
4,2020-04-28,26357,719,0,25638


In [8]:
%%bigquery
select date, avg(retail_and_rec) as retail_and_rec, avg(grocery_and_pharm) as grocery_and_pharm, avg(parks) as parks, avg(transit_stations) as transit_stations, avg(workplaces) as workplaces, avg(residential) as residential
from google_modeled.mobility_event g join google_modeled.location l
on g.location_id = l.id
where sub_region_1 = 'Texas'
group by date, country_region
order by date desc
limit 5

,date,retail_and_rec,grocery_and_pharm,parks,transit_stations,workplaces,residential
0,2020-04-26,-24.663043,-10.554348,-0.098039,-26.265957,-26.939189,9.574468
1,2020-04-25,-26.803922,-6.396040,0.862069,-23.816327,-21.313725,11.117647
2,2020-04-24,-25.866667,-8.380952,1.450980,-22.626263,-33.974359,17.024096
3,2020-04-23,-23.061224,-6.291667,11.166667,-20.826531,-33.497462,15.370787
4,2020-04-22,-27.144330,-11.043011,-11.533333,-21.707071,-34.193878,17.102273


cross dataset queries

In [9]:
%%bigquery
select r.date, county_name as county, r.confirmed_cases as confirmed, r.deaths
from covid19_usafacts_modeled.reports r 
join covid19_usafacts_modeled.loc_state_Beam_DF s on s.state_fips_code = r.state_fips_code
join covid19_usafacts_modeled.loc_county c on c.county_fips_code = r.county_fips_code
left join
 (select j.date, city_county as county, sum(j.confirmed) as confirmed, sum(j.deaths) as deaths
  from covid19_jhu_csse_modeled.reports j 
  join covid19_jhu_csse_modeled.us_location_id_Beam_DF l on j.location_id = l.id
  where province_state = 'Texas'
  group by date, city_county) j1 on r.date = j1.date and j1.county = county_name
where state = 'Texas'
order by r.date desc, county_name
limit 12

,date,county,confirmed,deaths
0,2020-05-02,Anderson County,32,0
1,2020-05-02,Andrews County,20,0
2,2020-05-02,Angelina County,55,0
3,2020-05-02,Aransas County,2,0
4,2020-05-02,Archer County,0,0
5,2020-05-02,Armstrong County,2,0
6,2020-05-02,Atascosa County,19,1
7,2020-05-02,Austin County,13,0
8,2020-05-02,Bailey County,0,0
9,2020-05-02,Bandera County,6,0


#### create views

In [10]:
dataset_id = "covid19_views" #`spry-cosine-266801.

In [11]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset 'spry-cosine-266801:covid19_views'
already exists.


In [12]:
%%bigquery
create or replace view covid19_views.v_world_cases as
select date, country_region, sum(confirmed) as confirmed, sum(deaths) as deaths, sum(recovered) as recovered, sum(active) as active
from `spry-cosine-266801.covid19_jhu_csse_modeled.reports` r 
join `spry-cosine-266801.covid19_jhu_csse_modeled.location_id_Beam_DF` l
on r.location_id = l.id
group by date, country_region
order by date desc

""


In [13]:
%%bigquery
create or replace view covid19_views.v_world_mobility as
select date, country_region, avg(retail_and_rec) as retail_and_rec, avg(grocery_and_pharm) as grocery_and_pharm, avg(parks) as parks, avg(transit_stations) as transit_stations, avg(workplaces) as workplaces, avg(residential) as residential
from `spry-cosine-266801.google_modeled.mobility_event` g join `spry-cosine-266801.google_modeled.location` l
on g.location_id = l.id
group by date, country_region
order by date desc

""


In [14]:
%%bigquery
create or replace view covid19_views.v_world_cross as
select j1.date, j1.country_region, confirmed, workplaces, residential from 
(select r.date, l.country_region, sum(confirmed) as confirmed
from `spry-cosine-266801.covid19_jhu_csse_modeled.reports` r 
join `spry-cosine-266801.covid19_jhu_csse_modeled.location_id_Beam_DF` l on r.location_id = l.id
group by r.date, country_region) j1
join
(select g.date, gl.country_region, avg(workplaces) as workplaces, avg(residential) as residential 
from `spry-cosine-266801.google_modeled.mobility_event` g
join `spry-cosine-266801.google_modeled.location` gl on gl.id = g.location_id
group by g.date, country_region) j2
on j1.date = j2.date and j1.country_region = j2.country_region
order by date desc, country_region

""


In [15]:
%%bigquery
create or replace view covid19_views.v_us_mobility as
select date, avg(retail_and_rec) as retail_and_rec, avg(grocery_and_pharm) as grocery_and_pharm, avg(parks) as parks, avg(transit_stations) as transit_stations, avg(workplaces) as workplaces, avg(residential) as residential
from `spry-cosine-266801.google_modeled.mobility_event` g 
join `spry-cosine-266801.google_modeled.location` l
on g.location_id = l.id
where country_region_code = 'US' 
group by date
order by date desc

""


In [16]:
%%bigquery
create or replace view covid19_views.v_us_cases_by_state as
select j1.date, j1.state, j1.confirmed, j1.deaths from
(select date, province_state as state, sum(j.confirmed) as confirmed, sum(j.deaths) as deaths
from `spry-cosine-266801.covid19_jhu_csse_modeled.reports` j 
join `spry-cosine-266801.covid19_jhu_csse_modeled.us_location_id_Beam_DF` l on j.location_id = l.id
group by date, province_state) j1
join
(select date, state, sum(confirmed_cases) as confirmed, sum(deaths) as deaths
from `spry-cosine-266801.covid19_usafacts_modeled.reports` r 
join `spry-cosine-266801.covid19_usafacts_modeled.loc_state_Beam_DF` s on s.state_fips_code = r.state_fips_code
group by date, state) j2 on j1.date=j2.date and j1.state=j2.state
order by date desc, state

""


In [17]:
%%bigquery
create or replace view covid19_views.v_tx_mobility as
select date, avg(retail_and_rec) as retail_and_rec, avg(grocery_and_pharm) as grocery_and_pharm, avg(parks) as parks, avg(transit_stations) as transit_stations, avg(workplaces) as workplaces, avg(residential) as residential
from `spry-cosine-266801.google_modeled.mobility_event` g 
join `spry-cosine-266801.google_modeled.location` l
on g.location_id = l.id
where sub_region_1 = 'Texas'
group by date, country_region
order by date desc

""


In [18]:
%%bigquery
create or replace view covid19_views.v_tx_cases_by_county as
select r.date, county_name as county, r.confirmed_cases as confirmed, r.deaths
from `spry-cosine-266801.covid19_usafacts_modeled.reports` r 
join `spry-cosine-266801.covid19_usafacts_modeled.loc_state_Beam_DF` s on s.state_fips_code = r.state_fips_code
join `spry-cosine-266801.covid19_usafacts_modeled.loc_county` c on c.county_fips_code = r.county_fips_code
left join
(select j.date, city_county as county, sum(j.confirmed) as confirmed, sum(j.deaths) as deaths
from `spry-cosine-266801.covid19_jhu_csse_modeled.reports` j 
join `spry-cosine-266801.covid19_jhu_csse_modeled.us_location_id_Beam_DF` l on j.location_id = l.id
where province_state = 'Texas'
group by date, city_county) j1 on r.date = j1.date and j1.county = county_name
where state = 'Texas'
order by r.date desc, county_name

""
